# Thread Scraping(You will get 'text6.csv' by running this code)

In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import time

import requests
from bs4 import BeautifulSoup

In [2]:
import os
print(os.getcwd())

In [1]:
PATH = r"C:\Users\Doheon Han\PythonWorkspace\UAV\chromedriver.exe"
url = 'https://discuss.ardupilot.org/c/log-analysis-and-tuning/log-analysis-copter/41'

# for this code, you need to install chrome driver
driver = webdriver.Chrome(PATH)
driver.get(url)

In [77]:
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [78]:
posts = driver.page_source

In [79]:
driver.close()

In [80]:
html = BeautifulSoup(posts, "html.parser")

In [81]:
L = []                             # Thread addresses
for s in html.select('span'):
    if s.get('class') == ['link-top-line']:
        for link in s.select('a'):
            if not "https://discuss.ardupilot.org" in link.get('href'):
                L.append("https://discuss.ardupilot.org"+link.get('href'))

R = []                             # numbers of replies
for td in html.select('td'):
    for number in td.select('button'):
        for span in number.select('span'):
            R.append(span.get_text())

print(len(L),len(R))

891 891


In [83]:
S = []   # Total number of comments
for r in R:
    r = int(r)
    S.append(r)
print(sum(S), S)

3361 [1, 0, 3, 1, 15, 3, 3, 22, 8, 12, 7, 14, 30, 9, 8, 9, 11, 4, 11, 1, 11, 4, 6, 2, 24, 3, 7, 1, 2, 8, 4, 2, 2, 1, 1, 0, 4, 3, 3, 1, 21, 0, 9, 2, 2, 6, 1, 1, 0, 0, 9, 2, 4, 0, 2, 7, 11, 2, 3, 13, 5, 8, 7, 4, 0, 4, 1, 1, 14, 0, 0, 0, 1, 18, 18, 6, 13, 0, 1, 3, 2, 2, 0, 4, 7, 4, 0, 2, 20, 2, 6, 9, 0, 11, 0, 4, 9, 23, 0, 11, 4, 3, 0, 1, 6, 4, 14, 4, 2, 0, 2, 6, 3, 6, 1, 1, 8, 0, 9, 4, 2, 15, 0, 0, 4, 3, 5, 0, 2, 2, 0, 10, 8, 3, 1, 5, 5, 4, 5, 3, 0, 0, 4, 0, 4, 3, 3, 4, 2, 0, 0, 5, 4, 0, 1, 4, 2, 1, 0, 2, 1, 0, 3, 6, 0, 4, 7, 0, 8, 0, 1, 7, 4, 0, 3, 0, 1, 4, 5, 3, 3, 3, 10, 0, 0, 2, 0, 2, 1, 2, 4, 0, 2, 6, 2, 2, 5, 4, 0, 2, 0, 4, 7, 1, 0, 2, 11, 1, 0, 4, 0, 1, 0, 13, 0, 0, 1, 4, 4, 2, 2, 1, 5, 2, 1, 2, 2, 1, 0, 2, 1, 9, 0, 2, 0, 1, 0, 8, 12, 6, 0, 0, 2, 1, 1, 1, 3, 1, 1, 2, 3, 1, 1, 1, 2, 0, 1, 1, 0, 2, 1, 1, 3, 3, 3, 1, 2, 2, 3, 0, 4, 0, 1, 3, 1, 9, 1, 2, 1, 0, 2, 5, 3, 0, 0, 1, 2, 7, 1, 3, 6, 0, 2, 0, 0, 0, 0, 6, 2, 3, 6, 3, 1, 1, 4, 2, 9, 0, 5, 9, 6, 0, 3, 2, 2, 2, 1, 0, 1, 1, 4, 0, 0

In [84]:
# Getting title of thread
def get_title(url):
    page = requests.get(url)
    status = page.status_code

    text = BeautifulSoup(page.content)
    title = text.title
    title = title.get_text()
    title = title.split(' - ')

    if status == 200:   
        return title
    else:
        return False

# Getting user id of comment. This is for getting user stats later.    
def get_user(text):
    userlinks=[]
    for part in text.select('.crawler-post-meta'):
        clock = str(part.select('.post-time'))[45:55]    
        for a in part.select('a'):
            userlinks.append(a.get('href')+clock)
    return userlinks

# Getting userlink of comment
def get_link(text):
    Link=[]
    Time=[]
    for part in text.select('.crawler-post-meta'):
        tim = str(part.select('.post-time'))[45:55]
        Time.append(tim)

    for post in text.select('.post'):
        Link.append("NOJAM"+post.get_text()[:20])
        for link in post.select('a'):
            if link.get('href'):
                if link.get('href')[0] != '/':
                    Link[-1] = link.get('href')
                    break

    for i in range(len(Link)):
        Link[i] = Link[i] + Time[i]
    return Link

# Getting text of comment
def get_post(text):
    for d in text.select('.post'):
        for s in d.select('aside'):
            s.replace_with('*')
    for d in text.select('.post'):
        for s in d.select('.lightbox-wrapper'):
            s.replace_with('*')
    for d in text.select('.post'):
        for s in d.select('p'):
             if "[quote" in s.get_text():
                    s.replace_with('*')
    for d in text.select('.post'):
        for s in d.select('p'):
             if "/quote]"in s.get_text():
                    if not "quote]\n" in s.get_text():
                        s.replace_with('*')
    for d in text.select('.post'):
        for s in d.select('ol'):
             if "/quote]"in s.get_text():
                    if not "quote]\n" in s.get_text():
                        s.replace_with('*')
    for d in text.select('.post'):
        for s in d.select('pre'):
            s.replace_with('*')            
    for d in text.select('.post'):
        for s in d.select('p'):
            for b in s.select('a'):
                b.replace_with('*')
    A=[]
    B=[]
    for d in text.select('.post'):
        for s in d:
            s = s.get_text()
            if s != '\n':
                if s != '':
                    s = s.replace("\n", " ")
                    if "[/quote]" in s:
                        s = s.split("[/quote]")[1]
                    if "[quote" in s:
                        s = s.split("[quote")[1]
                    if s not in str(A):
                        B.append(s)
        if len(B) != 0:
            for m in range(len(B)):
                g = " ".join(B)
            A.append(g)
            B=[]
        else:     
            g = "*"
            A.append(g)
#             continue
    return A

# Getting posted data of comment
def get_date(text):
    c_Dates = []
    for time in text.select('time'):
        value = time.get('datetime')
        c_Dates.append(value)
        
    return c_Dates

def JaccardSimilarity(t, a):
    list_inp1 = t.split()
    list_inp2 = a.split()
    mom = set(list_inp1).union(set(list_inp2))
    son = set(list_inp1).intersection(set(list_inp2))
    return len(son)/len(mom)

In [73]:
# Printing for test
Userlinks=[]
Links=[]
all_c_Dates=[]
for i in range(len(L)):
    url = L[i]
    page = requests.get(url)
    text = BeautifulSoup(page.content)
    
    addnum = int(R[i])+1
    num = int(addnum//20.000001)
    X = []
    for n in range(0,num+1):
        q = n*20+6
        w = str(q)
        X.append('/'+ w)
        
    title = get_title(url)
    print("{0} of {1}: ".format(i+1, len(L)), title)
    print("-"*100)
    
    Y=X
    for y in Y:
        url_m1 = url + y
        page1 = requests.get(url_m1)
        text1 = BeautifulSoup(page1.content)
        t_Link = get_link(text1)
        if y == '/6':
            Link = t_Link
        else:
            for tlink in t_Link:
                for link in Link:
                    if link == tlink:
                        break
                if link == tlink:
                    continue
                Link.append(tlink)
    Links.append(Link)
    
    for x in X:
        url_m = url + x
        page = requests.get(url_m)
        text = BeautifulSoup(page.content)
        U = get_user(text)
        A = get_post(text)
        a_Dates = get_date(text)
        if x == '/6':
            E = U
            T = A
            c_Dates = a_Dates
        else:
            for u in U:
                for e in E:
                    if e == u:
                        break
                if e == u:
                    continue
                E.append(u)                          
            
            for a in A:
                for t in T:
                    if JaccardSimilarity(t, a) > 0.9:
                        break
                if JaccardSimilarity(t, a) > 0.9:
                    continue
                T.append(a)

            for a_date in a_Dates:
                for c_date in c_Dates:
                    if c_date == a_date:
                        break
                if c_date == a_date:
                    continue
                c_Dates.append(a_date)
    all_c_Dates.append(c_Dates)
    Userlinks.append(E)
            
    print(T, len(T), type(T))
    print("-"*100)
    print(c_Dates, len(c_Dates))
    print("-"*100)
    print(len(Link), Link)
    print("-"*100)
    print(len(E), E)
    print("-"*100)    

88 of 891:  ['When autotune becomes a crash test', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
['Hi, I’m a fan of ArduCopter for a number of months now, I’ve trying to build a copter dedicated to photogrammetry on an Alien GF680 frame. Step by steps reading the wiki and the forum I tried to manage to set the chute, the camera but this time I \'m lost trying to find an explanation on an autotune crash. Any help would be much appreciated to work out what went wrong. Hardware & Software Pixhack CUAV Version 2.8 Firmware Version V3.4.2 680 mm diag frame Sunnysky X4110S  580KV 40 A ESC DYS with SimonK, 4S 10000mAh, 15,5 " wooden propellers The chute and its launcher are on and the a gimbal is tied. Total weight 3,500 Kg Taranis Tx/Rx FrSky Crash Description Flown well previously in stabilise and alt-hold mode. No desync on each  esc-motors noticed during ground test. See this video :

90 of 891:  ['Assign a button to tag an event in logs', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
['Is there a parameter or something that can be assigned to a button\\switch that when triggered just tags an event in the log?  I’ve had a couple of fail safes (no radio signal) occur but they aren’t always in the log. But I’d like to be able to mark when an event occurs so I can easily analyze it.  Thoughts?', 'RC in is recorded for all channels. Just move or toggle a switch on an unused channel, then you’ll find it in log easily.', 'Hmm I’m on the Skyviper and have buttons, I might have to figure something else out. Thanks!'] 3 <class 'list'>
----------------------------------------------------------------------------------------------------
['2018-02-24T20:44:36Z', '2018-02-24T22:41:19Z', '2018-02-25T01:54:24Z'] 3
---------------------------------------------------------------

In [85]:
# Making whole dataset as CSV file.
import pandas as pd
df1 = pd.DataFrame({"Title": [""], "Contents": [""]})

Userlinks=[]
Links=[]
all_c_Dates=[]
for i in range(len(L)):
    url = L[i]
    page = requests.get(url)
    text = BeautifulSoup(page.content)
    
    addnum = int(R[i])+1
    num = int(addnum//20.000001)
    X = []
    for n in range(0,num+1):
        q = n*20+6
        w = str(q)
        X.append('/'+ w)
        
    title = get_title(url)
    print("{0} of {1}: ".format(i+1, len(L)), title)
    print("-"*100)
    
    Y=X
    for y in Y:
        url_m1 = url + y
        page1 = requests.get(url_m1)
        text1 = BeautifulSoup(page1.content)
        t_Link = get_link(text1)
        if y == '/6':
            Link = t_Link
        else:
            for tlink in t_Link:
                for link in Link:
                    if link == tlink:
                        break
                if link == tlink:
                    continue
                Link.append(tlink)
    Links.append(Link)
    
    for x in X:
        url_m = url + x
        page = requests.get(url_m)
        text = BeautifulSoup(page.content)
        U = get_user(text)
        A = get_post(text)
        a_Dates = get_date(text)
        if x == '/6':
            E = U
            T = A
            c_Dates = a_Dates
        else:
            for u in U:
                for e in E:
                    if e == u:
                        break
                if e == u:
                    continue
                E.append(u)                          
            
            for a in A:
                for t in T:
                    if JaccardSimilarity(t, a) > 0.9:
                        break
                if JaccardSimilarity(t, a) > 0.9:
                    continue
                T.append(a)

            for a_date in a_Dates:
                for c_date in c_Dates:
                    if c_date == a_date:
                        break
                if c_date == a_date:
                    continue
                c_Dates.append(a_date)
    all_c_Dates.append(c_Dates)
    Userlinks.append(E)
    
    title_s = str(title[0])
    for j in range(1, len(title[:-2])):
        title_s = str(title_s)+' '+str(title[j])
    
    df2 = pd.DataFrame({"Title": [title_s], "Contents": [T]}, index=[i+1])
    df1 = pd.concat([df1, df2])

    time.sleep(0.2)

1 of 891:  ['About the Copter Logs -- deprecated category', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
2 of 891:  ['Attention: Please download PIXHAWK logs from the SD card', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
3 of 891:  ['Extracting pitch, roll & yaw data for image mosaic', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
4 of 891:  ['Would like to know about PID tuning of drones in detail', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
5 of 891:  ['Tuning large 36Kg X8 drone', 'seeking advice', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
-------

41 of 891:  ['Log Crash Analysis Help', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
42 of 891:  ['Acro settings for racing/freestyle ~220mm quad', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
43 of 891:  ['Need help! Severe shaking on roll axis', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
44 of 891:  ['Log Help: Work Solo fell out of the sky', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
45 of 891:  ['AC 3.6.3', 'Drone crash during normal flight (Loiter)', 'why? (inc. LOG)', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
---------------------------------

79 of 891:  ['Copter rolls right upon forward flight, left on backward flight', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
80 of 891:  ['My Vibe is looking pretty bad, what can I do to diagnose?', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
81 of 891:  ['Compass Offsets HIGH // external + internal compass in use', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
82 of 891:  ['Unknown event 62', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
83 of 891:  ['Solo logs ending early during big battery testing', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
--------

119 of 891:  ['3.5.0-rc8 "FlipOver" during TakeOff', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
120 of 891:  ['Vehicle out of control after few minutes', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
121 of 891:  ['Crash in Loiter Mode, How can I avoid new crashes in this mode?', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
122 of 891:  ['Strange Pitch-axis Oscillation', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
123 of 891:  ['Weird Behavior right after takeoff and right before landing', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
-------------------

158 of 891:  ['Erratic Behavior while flying', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
159 of 891:  ['ESC or Motor issue? Please help', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
160 of 891:  ['Pixhawk fails to log any data', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
161 of 891:  ['Fine tuning 210 race quad', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
162 of 891:  ['Unable to determine cause of RCOU.C2 spike', 'with video', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------

...

715 of 891:  ['In-Flight Tuning BUG', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
716 of 891:  ['Strange altitude behavior almost lead to crash', 'help needed', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
717 of 891:  ['Waypoint Problem', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
718 of 891:  ['Pixhawk crash went nuts during landing appr, have logs/video', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
719 of 891:  ['Motors Cutting in and Out in Loiter Mode', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
-------------------------------------------------

756 of 891:  ['GPS lock', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
757 of 891:  ['Too many crashes', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
758 of 891:  ['Droneshare and Logs', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
759 of 891:  ['Possible to start logging without arming?', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
760 of 891:  ['Would it make sense to revert back to 2.9 briefly?', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
761 of 891:

797 of 891:  ["Can't access Datafalsh logs in Terminal or Flight Data", 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
798 of 891:  ['RTL too high', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
799 of 891:  ['Quadcopter Motors Spin unevenly', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
800 of 891:  ['Recovering .param from .log file', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
801 of 891:  ['Copter lost control during Auto Mode', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
---------------------------------------------------------------------------------

837 of 891:  ['Missions Waypoint problems', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
838 of 891:  ['Dataflash fails', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
839 of 891:  ['Unexplained Mission behaviour..possible bug', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
840 of 891:  ['Quad tilting to the left', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
841 of 891:  ['Best way to manually correct Accelerometer scaling/offsets?', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
-------------------------------------------------------------------------------

878 of 891:  ['Flyaway when RTL engaged; over 30mph on landing?', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
879 of 891:  ['APM Up side down relative to the frame', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
880 of 891:  ['Autotune values and loiter', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
881 of 891:  ['Alternatives to Mission Planner for plotting log files?', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
----------------------------------------------------------------------------------------------------
882 of 891:  ['Interpreting THRIn & ThrOut on a graph', 'Copter Logs -- deprecated', 'ArduPilot Discourse']
-----------------------------------------

In [89]:
df1

,Title,Contents
0,,
1,About the Copter Logs -- deprecated category,"[Do not post here, please post in the *, You s..."
2,Attention: Please download PIXHAWK logs from t...,"[In order to help users better, I need readabl..."
3,"Extracting pitch, roll & yaw data for image mo...","[Aloha, I’m a grad student at the university o..."
4,Would like to know about PID tuning of drones ...,[I would like to know more about PID. The math...
...,...,...
887,Tuning Items explained,[Does anyone know of a good tutorial that can ...
888,Setting Throttle Mid,[As mentioned in this link: * at what altitude...
889,"3DR Y6 crash, failsafe RTL flip, logfile analy...",[Hoping to get some expertise on deciphering m...
890,Dataflash logging and performance,"[Hello, I just read the documentation on dataf..."


In [90]:
df = df1.drop(index=[0])
df

,Title,Contents
1,About the Copter Logs -- deprecated category,"[Do not post here, please post in the *, You s..."
2,Attention: Please download PIXHAWK logs from t...,"[In order to help users better, I need readabl..."
3,"Extracting pitch, roll & yaw data for image mo...","[Aloha, I’m a grad student at the university o..."
4,Would like to know about PID tuning of drones ...,[I would like to know more about PID. The math...
5,Tuning large 36Kg X8 drone seeking advice,[I have been working on the construction of a ...
...,...,...
887,Tuning Items explained,[Does anyone know of a good tutorial that can ...
888,Setting Throttle Mid,[As mentioned in this link: * at what altitude...
889,"3DR Y6 crash, failsafe RTL flip, logfile analy...",[Hoping to get some expertise on deciphering m...
890,Dataflash logging and performance,"[Hello, I just read the documentation on dataf..."


In [91]:
df.to_csv("post_text1.csv", index=False, encoding="UTF8")

In [92]:
pd.read_csv("post_text1.csv", encoding="UTF8")

,Title,Contents
0,About the Copter Logs -- deprecated category,"['Do not post here, please post in the *', 'Yo..."
1,Attention: Please download PIXHAWK logs from t...,"['In order to help users better, I need readab..."
2,"Extracting pitch, roll & yaw data for image mo...","['Aloha, I’m a grad student at the university ..."
3,Would like to know about PID tuning of drones ...,['I would like to know more about PID. The mat...
4,Tuning large 36Kg X8 drone seeking advice,['I have been working on the construction of a...
...,...,...
886,Tuning Items explained,['Does anyone know of a good tutorial that can...
887,Setting Throttle Mid,['As mentioned in this link: * at what altitud...
888,"3DR Y6 crash, failsafe RTL flip, logfile analy...",['Hoping to get some expertise on deciphering ...
889,Dataflash logging and performance,"['Hello, I just read the documentation on data..."


# text5.csv(=Post|Title|Replies|Views|Comment#|Query|Contents|Len|Days|Links)

In [117]:
# Deep Copy
import copy
mf = copy.deepcopy(df)
mf

,Title,Contents
1,About the Copter Logs -- deprecated category,"[Do not post here, please post in the *, You s..."
2,Attention: Please download PIXHAWK logs from t...,"[In order to help users better, I need readabl..."
3,"Extracting pitch, roll & yaw data for image mo...","[Aloha, I’m a grad student at the university o..."
4,Would like to know about PID tuning of drones ...,[I would like to know more about PID. The math...
5,Tuning large 36Kg X8 drone seeking advice,[I have been working on the construction of a ...
...,...,...
887,Tuning Items explained,[Does anyone know of a good tutorial that can ...
888,Setting Throttle Mid,[As mentioned in this link: * at what altitude...
889,"3DR Y6 crash, failsafe RTL flip, logfile analy...",[Hoping to get some expertise on deciphering m...
890,Dataflash logging and performance,"[Hello, I just read the documentation on dataf..."


In [118]:
Views = []
for td in html.select('td'):
    for number in td.select('span'):
        if number.get('class') == ['number']:
            if number.get('title'):
                Views.append(number.get('title').split()[5])
print(len(Views),Views)

891 ['1444', '2056', '1136', '812', '5331', '891', '1078', '995', '811', '9885', '1536', '2361', '3532', '1893', '2097', '2468', '1667', '6054', '3799', '933', '3555', '974', '2193', '1585', '3903', '4828', '963', '779', '824', '1958', '1021', '736', '656', '1529', '555', '537', '782', '826', '1054', '580', '1336', '563', '2480', '604', '581', '961', '851', '555', '561', '689', '1792', '495', '1028', '375', '1000', '686', '941', '583', '415', '843', '690', '789', '1017', '2322', '394', '848', '556', '496', '1145', '432', '532', '464', '1378', '1136', '2581', '900', '1043', '503', '650', '554', '2147', '711', '543', '832', '873', '691', '442', '1288', '2168', '564', '936', '1556', '525', '1535', '607', '1207', '2186', '2462', '688', '1461', '1476', '1076', '594', '700', '839', '968', '2015', '996', '902', '1117', '1486', '1063', '837', '1084', '1678', '763', '1487', '796', '1677', '1048', '1430', '2188', '734', '675', '816', '1203', '1221', '702', '1065', '970', '733', '1093', '1511', '

In [120]:
import pandas as pd
mf1 = pd.DataFrame({"Post": [""], "Title": [""], "Replies": [""], "Views": [""], "Comment#": [""], "Query": [""],
                    "Contents": [""]})

for i in range(1, len(contents)+1):
    contents_list=[]
    if len(contents[i]) <= 1:
        mf2 = pd.DataFrame({"Post": [i], "Title": [title[i]], "Replies": [R[i-1]], "Views": [Views[i-1]], 
                            "Comment#": [0], "Query": [contents[i][0]], "Contents": [""], })
        mf1 = pd.concat([mf1, mf2])
    if len(contents[i]) == 2:
        contents_list.append(contents[i][0])
        contents_list.append(contents[i][1])
        mf2 = pd.DataFrame({"Post": [i], "Title": [title[i]], "Replies": [R[i-1]], "Views": [Views[i-1]], 
                            "Comment#": [1], "Query": [contents[i][0]], "Contents": [contents[i][1]], })
        mf1 = pd.concat([mf1, mf2])
    if len(contents[i]) > 2:
        for j in range(1, len(contents[i])):
            contents_list=[]
            contents_list.append(contents[i][0])
            contents_list.append(contents[i][j])
            mf2 = pd.DataFrame({"Post": [i], "Title": [title[i]], "Replies": [R[i-1]], "Views": [Views[i-1]], 
                                "Comment#": [j], "Query": [contents[i][0]], "Contents": [contents[i][j]], })
            mf1 = pd.concat([mf1, mf2])

In [121]:
numlist = list(range(len(mf1)))
mf1 = mf1.set_index(pd.Index(numlist))
mf = mf1.drop(index=[0])
mf

,Post,Title,Replies,Views,Comment#,Query,Contents
1,1,About the Copter Logs -- deprecated category,1,1444,1,"Do not post here, please post in the *",You should probably change this
2,2,Attention: Please download PIXHAWK logs from t...,0,2056,0,"In order to help users better, I need readable...",
3,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,1,"Aloha, I’m a grad student at the university of...",I’m interested in this as well. I’ll be doing...
4,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,2,"Aloha, I’m a grad student at the university of...",Did you figure out how/what logs to use to get...
5,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,3,"Aloha, I’m a grad student at the university of...",I am having the same problems at the moment Ha...
...,...,...,...,...,...,...,...
3467,888,Setting Throttle Mid,2,252,1,As mentioned in this link: * at what altitude ...,Not sure why you think altitude matters. Anyth...
3468,888,Setting Throttle Mid,2,252,2,As mentioned in this link: * at what altitude ...,As they didn’t mentioned any altitude thats wh...
3469,889,"3DR Y6 crash, failsafe RTL flip, logfile analy...",1,287,1,Hoping to get some expertise on deciphering my...,I should add that the copter is physically and...
3470,890,Dataflash logging and performance,1,222,1,"Hello, I just read the documentation on datafl...","Hi bds, The easy way is simply to observe perf..."


In [122]:
mf['Len'] = [len(content) for content in mf['Contents']]

In [123]:
mf

,Post,Title,Replies,Views,Comment#,Query,Contents,Len
1,1,About the Copter Logs -- deprecated category,1,1444,1,"Do not post here, please post in the *",You should probably change this,31
2,2,Attention: Please download PIXHAWK logs from t...,0,2056,0,"In order to help users better, I need readable...",,0
3,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,1,"Aloha, I’m a grad student at the university of...",I’m interested in this as well. I’ll be doing...,278
4,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,2,"Aloha, I’m a grad student at the university of...",Did you figure out how/what logs to use to get...,79
5,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,3,"Aloha, I’m a grad student at the university of...",I am having the same problems at the moment Ha...,183
...,...,...,...,...,...,...,...,...
3467,888,Setting Throttle Mid,2,252,1,As mentioned in this link: * at what altitude ...,Not sure why you think altitude matters. Anyth...,105
3468,888,Setting Throttle Mid,2,252,2,As mentioned in this link: * at what altitude ...,As they didn’t mentioned any altitude thats wh...,100
3469,889,"3DR Y6 crash, failsafe RTL flip, logfile analy...",1,287,1,Hoping to get some expertise on deciphering my...,I should add that the copter is physically and...,704
3470,890,Dataflash logging and performance,1,222,1,"Hello, I just read the documentation on datafl...","Hi bds, The easy way is simply to observe perf...",598


In [124]:
print(all_c_Dates)

[['2013-09-18T08:32:59Z', '2016-04-20T15:56:12Z'], ['2014-03-13T19:08:17Z'], ['2015-12-22T01:31:30Z', '2015-12-23T04:42:15Z', '2016-06-11T18:39:53Z', '2022-07-04T08:32:12Z'], ['2016-05-06T09:38:45Z', '2022-06-17T14:37:23Z'], ['2016-03-23T16:37:03Z', '2016-03-28T07:32:34Z', '2016-03-28T11:41:53Z', '2016-03-28T14:10:25Z', '2016-03-29T08:02:21Z', '2016-03-29T08:03:15Z', '2016-03-30T15:25:27Z', '2016-04-08T19:29:34Z', '2016-04-11T17:12:56Z', '2016-04-22T02:16:00Z', '2016-05-08T10:40:19Z', '2016-05-10T16:54:31Z', '2018-09-23T13:50:17Z', '2021-07-20T06:38:52Z', '2022-04-24T11:01:28Z', '2022-04-24T16:11:52Z'], ['2014-02-10T06:30:35Z', '2014-02-11T01:51:50Z', '2022-02-15T08:54:19Z', '2022-02-15T09:30:05Z'], ['2016-03-15T20:50:50Z', '2021-11-29T05:37:45Z', '2021-11-29T22:12:30Z', '2021-11-30T01:53:06Z'], ['2014-07-12T16:41:50Z', '2014-07-12T18:02:10Z', '2014-07-13T02:02:50Z', '2014-07-13T06:25:35Z', '2014-07-13T11:55:32Z', '2014-07-13T12:51:12Z', '2014-07-13T13:12:18Z', '2021-10-16T17:55:03Z', 

In [125]:
atw=[]
for g in all_c_Dates:
    atw.append(len(g))
print(len(atw))
print(sum(atw))
print(atw)

891
4252
[2, 1, 4, 2, 16, 4, 4, 23, 9, 13, 8, 15, 31, 10, 9, 10, 12, 5, 12, 2, 12, 5, 7, 3, 25, 4, 8, 2, 3, 9, 5, 3, 3, 2, 2, 1, 5, 4, 4, 2, 22, 1, 10, 3, 3, 7, 2, 2, 1, 1, 10, 3, 5, 1, 3, 8, 12, 3, 4, 14, 6, 9, 8, 5, 1, 5, 2, 2, 15, 1, 1, 1, 2, 19, 19, 7, 14, 1, 2, 4, 3, 3, 1, 5, 8, 5, 1, 3, 21, 3, 7, 10, 1, 12, 1, 5, 10, 24, 1, 12, 5, 4, 1, 2, 7, 5, 15, 5, 3, 1, 3, 7, 4, 7, 2, 2, 9, 1, 10, 5, 3, 16, 1, 1, 5, 4, 6, 1, 3, 3, 1, 11, 9, 4, 2, 6, 6, 5, 6, 4, 1, 1, 5, 1, 5, 4, 4, 5, 3, 1, 1, 6, 5, 1, 2, 5, 3, 2, 1, 3, 2, 1, 4, 7, 1, 5, 8, 1, 9, 1, 2, 8, 5, 1, 4, 1, 2, 5, 6, 4, 4, 4, 11, 1, 1, 3, 1, 3, 2, 3, 5, 1, 3, 7, 3, 3, 6, 5, 1, 3, 1, 5, 8, 2, 1, 3, 12, 2, 1, 5, 1, 2, 1, 14, 1, 1, 2, 5, 5, 3, 3, 2, 6, 3, 2, 3, 3, 2, 1, 3, 2, 10, 1, 3, 1, 2, 1, 9, 13, 7, 1, 1, 3, 2, 2, 2, 4, 2, 2, 3, 4, 2, 2, 2, 3, 1, 2, 2, 1, 3, 2, 2, 4, 4, 4, 2, 3, 3, 4, 1, 5, 1, 2, 4, 2, 10, 2, 3, 2, 1, 3, 6, 4, 1, 1, 2, 3, 8, 2, 4, 7, 1, 3, 1, 1, 1, 1, 7, 3, 4, 7, 4, 2, 2, 5, 3, 10, 1, 6, 10, 7, 1, 4, 3, 3, 3, 2, 1

In [127]:
Dates=[]
for all_c_date in all_c_Dates:
    dates=[]
    for c_date in all_c_date:
        date=[]
        value = c_date
        year = value.split('-')[0]
        date.append(int(year))
        month = value.split('-')[1]
        date.append(int(month))
        day = value.split('-')[2][:2]
        date.append(int(day))
        dates.append(date)
    Dates.append(dates)
print(len(Dates), Dates)

891 [[[2013, 9, 18], [2016, 4, 20]], [[2014, 3, 13]], [[2015, 12, 22], [2015, 12, 23], [2016, 6, 11], [2022, 7, 4]], [[2016, 5, 6], [2022, 6, 17]], [[2016, 3, 23], [2016, 3, 28], [2016, 3, 28], [2016, 3, 28], [2016, 3, 29], [2016, 3, 29], [2016, 3, 30], [2016, 4, 8], [2016, 4, 11], [2016, 4, 22], [2016, 5, 8], [2016, 5, 10], [2018, 9, 23], [2021, 7, 20], [2022, 4, 24], [2022, 4, 24]], [[2014, 2, 10], [2014, 2, 11], [2022, 2, 15], [2022, 2, 15]], [[2016, 3, 15], [2021, 11, 29], [2021, 11, 29], [2021, 11, 30]], [[2014, 7, 12], [2014, 7, 12], [2014, 7, 13], [2014, 7, 13], [2014, 7, 13], [2014, 7, 13], [2014, 7, 13], [2021, 10, 16], [2021, 10, 16], [2021, 10, 17], [2021, 10, 17], [2021, 10, 18], [2021, 10, 18], [2021, 10, 18], [2021, 10, 18], [2021, 10, 19], [2021, 10, 19], [2021, 10, 25], [2021, 10, 25], [2021, 10, 29], [2021, 10, 29], [2021, 10, 29], [2021, 10, 29]], [[2014, 3, 5], [2014, 3, 5], [2014, 3, 5], [2014, 3, 5], [2014, 3, 6], [2014, 3, 23], [2014, 7, 9], [2014, 7, 12], [2021, 

In [128]:
da=[]
w=0
for dates in Dates:
    w+=len(dates)
    da.append(len(dates))
print(w)
print(4251-891)
print(da)
print(sum(da))
print(len(da))

4252
3360
[2, 1, 4, 2, 16, 4, 4, 23, 9, 13, 8, 15, 31, 10, 9, 10, 12, 5, 12, 2, 12, 5, 7, 3, 25, 4, 8, 2, 3, 9, 5, 3, 3, 2, 2, 1, 5, 4, 4, 2, 22, 1, 10, 3, 3, 7, 2, 2, 1, 1, 10, 3, 5, 1, 3, 8, 12, 3, 4, 14, 6, 9, 8, 5, 1, 5, 2, 2, 15, 1, 1, 1, 2, 19, 19, 7, 14, 1, 2, 4, 3, 3, 1, 5, 8, 5, 1, 3, 21, 3, 7, 10, 1, 12, 1, 5, 10, 24, 1, 12, 5, 4, 1, 2, 7, 5, 15, 5, 3, 1, 3, 7, 4, 7, 2, 2, 9, 1, 10, 5, 3, 16, 1, 1, 5, 4, 6, 1, 3, 3, 1, 11, 9, 4, 2, 6, 6, 5, 6, 4, 1, 1, 5, 1, 5, 4, 4, 5, 3, 1, 1, 6, 5, 1, 2, 5, 3, 2, 1, 3, 2, 1, 4, 7, 1, 5, 8, 1, 9, 1, 2, 8, 5, 1, 4, 1, 2, 5, 6, 4, 4, 4, 11, 1, 1, 3, 1, 3, 2, 3, 5, 1, 3, 7, 3, 3, 6, 5, 1, 3, 1, 5, 8, 2, 1, 3, 12, 2, 1, 5, 1, 2, 1, 14, 1, 1, 2, 5, 5, 3, 3, 2, 6, 3, 2, 3, 3, 2, 1, 3, 2, 10, 1, 3, 1, 2, 1, 9, 13, 7, 1, 1, 3, 2, 2, 2, 4, 2, 2, 3, 4, 2, 2, 2, 3, 1, 2, 2, 1, 3, 2, 2, 4, 4, 4, 2, 3, 3, 4, 1, 5, 1, 2, 4, 2, 10, 2, 3, 2, 1, 3, 6, 4, 1, 1, 2, 3, 8, 2, 4, 7, 1, 3, 1, 1, 1, 1, 7, 3, 4, 7, 4, 2, 2, 5, 3, 10, 1, 6, 10, 7, 1, 4, 3, 3, 3, 2, 

In [129]:
print(Dates[:2])
for dates in Dates:
    for date in dates:
        print(dates)
        print(date)
        break
    break 

[[[2013, 9, 18], [2016, 4, 20]], [[2014, 3, 13]]]
[[2013, 9, 18], [2016, 4, 20]]
[2013, 9, 18]


In [130]:
import datetime
Days = []
for dates in Dates:
    if len(dates) > 1:
        begin = datetime.date(dates[0][0], dates[0][1], dates[0][2])
        for i in range(1, len(dates)):
            end = datetime.date(dates[i][0], dates[i][1], dates[i][2])
            days = (end-begin).days
            Days.append(days)
    else:
        Days.append(0)
print(len(Days), Days)

3471 [945, 0, 1, 172, 2386, 2233, 5, 5, 5, 6, 6, 7, 16, 19, 30, 46, 48, 914, 1945, 2223, 2223, 1, 2927, 2927, 2085, 2085, 2086, 0, 1, 1, 1, 1, 1, 2653, 2653, 2654, 2654, 2655, 2655, 2655, 2655, 2656, 2656, 2662, 2662, 2666, 2666, 2666, 2666, 0, 0, 0, 1, 18, 126, 129, 2760, 0, 0, 0, 9, 260, 260, 260, 260, 285, 1172, 1818, 1983, 0, 0, 0, 0, 0, 2, 1617, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 964, 964, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1005, 1005, 1, 2, 3, 3, 452, 584, 2564, 2564, 2568, 6, 8, 8, 10, 48, 48, 196, 1184, 0, 0, 3, 3, 796, 796, 798, 798, 811, 1, 2, 3, 5, 5, 5, 8, 8, 20, 1826, 2252, 69, 213, 430, 1380, 0, 0, 0, 1, 2, 3, 3, 3, 4, 4, 1113, 628, 1, 1, 1, 2, 2, 6, 6, 7, 7, 73, 2249, 0, 0, 5, 2160, 1, 1, 1, 1, 1159, 1257, 91, 540, 0, 1, 1, 2, 2, 4, 4, 4, 8, 11, 12, 12, 15, 16, 16, 21, 22, 81, 81, 82, 83, 83, 283, 283, 0, 713, 753, 0, 2, 2, 3, 10, 10, 69, 60, 12, 12, 0, 0, 0, 4, 4, 4, 5, 210, 1, 1, 1, 2, 0, 2, 0, 1, 5, 0, 0, 0, 0, 0, 168,

In [131]:
mf['Days'] = Days

In [132]:
mf

,Post,Title,Replies,Views,Comment#,Query,Contents,Len,Days
1,1,About the Copter Logs -- deprecated category,1,1444,1,"Do not post here, please post in the *",You should probably change this,31,945
2,2,Attention: Please download PIXHAWK logs from t...,0,2056,0,"In order to help users better, I need readable...",,0,0
3,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,1,"Aloha, I’m a grad student at the university of...",I’m interested in this as well. I’ll be doing...,278,1
4,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,2,"Aloha, I’m a grad student at the university of...",Did you figure out how/what logs to use to get...,79,172
5,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,3,"Aloha, I’m a grad student at the university of...",I am having the same problems at the moment Ha...,183,2386
...,...,...,...,...,...,...,...,...,...
3467,888,Setting Throttle Mid,2,252,1,As mentioned in this link: * at what altitude ...,Not sure why you think altitude matters. Anyth...,105,0
3468,888,Setting Throttle Mid,2,252,2,As mentioned in this link: * at what altitude ...,As they didn’t mentioned any altitude thats wh...,100,0
3469,889,"3DR Y6 crash, failsafe RTL flip, logfile analy...",1,287,1,Hoping to get some expertise on deciphering my...,I should add that the copter is physically and...,704,2
3470,890,Dataflash logging and performance,1,222,1,"Hello, I just read the documentation on datafl...","Hi bds, The easy way is simply to observe perf...",598,0


In [133]:
print(len(Links), Links)

891 [['https://discuss.ardupilot.org/c/arducopterT08:32:59Z', 'NOJAM\nYou should probablyT15:56:12Z'], ['NOJAM\nIn order to help usT19:08:17Z'], ['NOJAM\nAloha, I’m a grad sT01:31:30Z', 'NOJAM\nI’m interested in tT04:42:15Z', 'NOJAM\nDid you figure out T18:39:53Z', 'mailto:mk.aburezeq@gmail.comT08:32:12Z'], ['NOJAM\nI would like to knoT09:38:45Z', 'https://www.youtube.com/watch?v=UR0hOmjaHp0T14:37:23Z'], ['http://ardupilot.com/forum/viewtopic.php?f=80&t=11884T16:37:03Z', 'NOJAM\nHi,\nI think there iT07:32:34Z', 'NOJAM\nHow large is large.T11:41:53Z', 'http://forums.ardupilot.orgT14:10:25Z', 'NOJAM\nHi chuckntaylor,\nI T08:02:21Z', 'NOJAM\nHi Mike,\nLarge is nT08:03:15Z', 'NOJAM\nHi Leonardthall,\nThT15:25:27Z', 'NOJAM\nHi Leonard,\nScarilyT19:29:34Z', 'NOJAM\nHave you perform a T17:12:56Z', 'NOJAM\nI tried out the setT02:16:00Z', 'NOJAM\nI lower the number T10:40:19Z', 'NOJAM\nI wanted to pass alT16:54:31Z', 'NOJAM\nI’m facing a very sT13:50:17Z', 'NOJAM\nDid you found a solT06:38:52Z'

In [138]:
for i in range(len(Links)):
    if not len(Links[i]) == 1:
        Links[i].remove(Links[i][0])
print(Links)

[['NOJAM\nYou should probablyT15:56:12Z'], ['NOJAM\nIn order to help usT19:08:17Z'], ['NOJAM\nI’m interested in tT04:42:15Z', 'NOJAM\nDid you figure out T18:39:53Z', 'mailto:mk.aburezeq@gmail.comT08:32:12Z'], ['https://www.youtube.com/watch?v=UR0hOmjaHp0T14:37:23Z'], ['NOJAM\nHi,\nI think there iT07:32:34Z', 'NOJAM\nHow large is large.T11:41:53Z', 'http://forums.ardupilot.orgT14:10:25Z', 'NOJAM\nHi chuckntaylor,\nI T08:02:21Z', 'NOJAM\nHi Mike,\nLarge is nT08:03:15Z', 'NOJAM\nHi Leonardthall,\nThT15:25:27Z', 'NOJAM\nHi Leonard,\nScarilyT19:29:34Z', 'NOJAM\nHave you perform a T17:12:56Z', 'NOJAM\nI tried out the setT02:16:00Z', 'NOJAM\nI lower the number T10:40:19Z', 'NOJAM\nI wanted to pass alT16:54:31Z', 'NOJAM\nI’m facing a very sT13:50:17Z', 'NOJAM\nDid you found a solT06:38:52Z', 'NOJAM\nHi!\nI would like toT11:01:28Z', 'NOJAM\nThe solution is to T16:11:52Z'], ['NOJAM\n[quote=“Gregmaan”]HT01:51:50Z', 'NOJAM\nI try to map lakes T08:54:19Z', 'https://ardupilot.org/rover/docs/paramete

In [139]:
LLinks = []
for i in range(len(Links)):
    for j in range(len(Links[i])):
        if Links[i][j][:5] == 'NOJAM':
            LLinks.append(0)
        else:
            LLinks.append(1)
print(len(LLinks), LLinks)

3471 [0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0

In [140]:
mf['Link'] = LLinks
mf

,Post,Title,Replies,Views,Comment#,Query,Contents,Len,Days,Link
1,1,About the Copter Logs -- deprecated category,1,1444,1,"Do not post here, please post in the *",You should probably change this,31,945,0
2,2,Attention: Please download PIXHAWK logs from t...,0,2056,0,"In order to help users better, I need readable...",,0,0,0
3,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,1,"Aloha, I’m a grad student at the university of...",I’m interested in this as well. I’ll be doing...,278,1,0
4,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,2,"Aloha, I’m a grad student at the university of...",Did you figure out how/what logs to use to get...,79,172,0
5,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,3,"Aloha, I’m a grad student at the university of...",I am having the same problems at the moment Ha...,183,2386,1
...,...,...,...,...,...,...,...,...,...,...
3467,888,Setting Throttle Mid,2,252,1,As mentioned in this link: * at what altitude ...,Not sure why you think altitude matters. Anyth...,105,0,0
3468,888,Setting Throttle Mid,2,252,2,As mentioned in this link: * at what altitude ...,As they didn’t mentioned any altitude thats wh...,100,0,0
3469,889,"3DR Y6 crash, failsafe RTL flip, logfile analy...",1,287,1,Hoping to get some expertise on deciphering my...,I should add that the copter is physically and...,704,2,0
3470,890,Dataflash logging and performance,1,222,1,"Hello, I just read the documentation on datafl...","Hi bds, The easy way is simply to observe perf...",598,0,0


In [141]:
mf.to_csv("post_text5.csv", index=False, encoding="UTF8")

In [142]:
pd.read_csv("post_text5.csv", encoding="UTF8")

,Post,Title,Replies,Views,Comment#,Query,Contents,Len,Days,Link
0,1,About the Copter Logs -- deprecated category,1,1444,1,"Do not post here, please post in the *",You should probably change this,31,945,0
1,2,Attention: Please download PIXHAWK logs from t...,0,2056,0,"In order to help users better, I need readable...",NaN,0,0,0
2,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,1,"Aloha, I’m a grad student at the university of...",I’m interested in this as well. I’ll be doing...,278,1,0
3,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,2,"Aloha, I’m a grad student at the university of...",Did you figure out how/what logs to use to get...,79,172,0
4,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,3,"Aloha, I’m a grad student at the university of...",I am having the same problems at the moment Ha...,183,2386,1
...,...,...,...,...,...,...,...,...,...,...
3466,888,Setting Throttle Mid,2,252,1,As mentioned in this link: * at what altitude ...,Not sure why you think altitude matters. Anyth...,105,0,0
3467,888,Setting Throttle Mid,2,252,2,As mentioned in this link: * at what altitude ...,As they didn’t mentioned any altitude thats wh...,100,0,0
3468,889,"3DR Y6 crash, failsafe RTL flip, logfile analy...",1,287,1,Hoping to get some expertise on deciphering my...,I should add that the copter is physically and...,704,2,0
3469,890,Dataflash logging and performance,1,222,1,"Hello, I just read the documentation on datafl...","Hi bds, The easy way is simply to observe perf...",598,0,0


# text6.csv(=Post|Title|Replies|Views|Comment#|Query|Contents|Len|Days|Links|Visit|Read|Topic|Posts|Given|Received|Topic_C|Post_C)

In [143]:
mf

,Post,Title,Replies,Views,Comment#,Query,Contents,Len,Days,Link
1,1,About the Copter Logs -- deprecated category,1,1444,1,"Do not post here, please post in the *",You should probably change this,31,945,0
2,2,Attention: Please download PIXHAWK logs from t...,0,2056,0,"In order to help users better, I need readable...",,0,0,0
3,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,1,"Aloha, I’m a grad student at the university of...",I’m interested in this as well. I’ll be doing...,278,1,0
4,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,2,"Aloha, I’m a grad student at the university of...",Did you figure out how/what logs to use to get...,79,172,0
5,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,3,"Aloha, I’m a grad student at the university of...",I am having the same problems at the moment Ha...,183,2386,1
...,...,...,...,...,...,...,...,...,...,...
3467,888,Setting Throttle Mid,2,252,1,As mentioned in this link: * at what altitude ...,Not sure why you think altitude matters. Anyth...,105,0,0
3468,888,Setting Throttle Mid,2,252,2,As mentioned in this link: * at what altitude ...,As they didn’t mentioned any altitude thats wh...,100,0,0
3469,889,"3DR Y6 crash, failsafe RTL flip, logfile analy...",1,287,1,Hoping to get some expertise on deciphering my...,I should add that the copter is physically and...,704,2,0
3470,890,Dataflash logging and performance,1,222,1,"Hello, I just read the documentation on datafl...","Hi bds, The easy way is simply to observe perf...",598,0,0


In [144]:
Userlinks

[['https://discuss.ardupilot.org/u/systemT08:32:59Z',
  'https://discuss.ardupilot.org/u/leking2T15:56:12Z'],
 ['https://discuss.ardupilot.org/u/jschallT19:08:17Z'],
 ['https://discuss.ardupilot.org/u/josephjkT01:31:30Z',
  'https://discuss.ardupilot.org/u/seanhT04:42:15Z',
  'https://discuss.ardupilot.org/u/fireflyer451T18:39:53Z',
  'https://discuss.ardupilot.org/u/Mohd_AbuRezeqT08:32:12Z'],
 ['https://discuss.ardupilot.org/u/prash319T09:38:45Z',
  'https://discuss.ardupilot.org/u/StevieReyesT14:37:23Z'],
 ['https://discuss.ardupilot.org/u/chuckntaylorT16:37:03Z',
  'https://discuss.ardupilot.org/u/LeonardthallT07:32:34Z',
  'https://discuss.ardupilot.org/u/mbolandT11:41:53Z',
  'https://discuss.ardupilot.org/u/chuckntaylorT14:10:25Z',
  'https://discuss.ardupilot.org/u/LeonardthallT08:02:21Z',
  'https://discuss.ardupilot.org/u/LeonardthallT08:03:15Z',
  'https://discuss.ardupilot.org/u/chuckntaylorT15:25:27Z',
  'https://discuss.ardupilot.org/u/mbolandT19:29:34Z',
  'https://discus

In [146]:
for i in range(len(Userlinks)):    # 댓글 없는것, 즉 길이 1인 것 '0'으로 만들고 그 외에는 첫번째 글이 quesry이므로 삭제
    if not len(Userlinks[i]) == 1:
        Userlinks[i].remove(Userlinks[i][0])
    else:
        Userlinks[i][0] = '0'
print(Userlinks)

[['https://discuss.ardupilot.org/u/leking2T15:56:12Z'], ['0'], ['https://discuss.ardupilot.org/u/seanhT04:42:15Z', 'https://discuss.ardupilot.org/u/fireflyer451T18:39:53Z', 'https://discuss.ardupilot.org/u/Mohd_AbuRezeqT08:32:12Z'], ['https://discuss.ardupilot.org/u/StevieReyesT14:37:23Z'], ['https://discuss.ardupilot.org/u/LeonardthallT07:32:34Z', 'https://discuss.ardupilot.org/u/mbolandT11:41:53Z', 'https://discuss.ardupilot.org/u/chuckntaylorT14:10:25Z', 'https://discuss.ardupilot.org/u/LeonardthallT08:02:21Z', 'https://discuss.ardupilot.org/u/LeonardthallT08:03:15Z', 'https://discuss.ardupilot.org/u/chuckntaylorT15:25:27Z', 'https://discuss.ardupilot.org/u/mbolandT19:29:34Z', 'https://discuss.ardupilot.org/u/cookmangrT17:12:56Z', 'https://discuss.ardupilot.org/u/chuckntaylorT02:16:00Z', 'https://discuss.ardupilot.org/u/cookmangrT10:40:19Z', 'https://discuss.ardupilot.org/u/chuckntaylorT16:54:31Z', 'https://discuss.ardupilot.org/u/rohanrautT13:50:17Z', 'https://discuss.ardupilot.org

In [147]:
Ulinks=[]         
for i in range(len(Userlinks)):
    for j in range(len(Userlinks[i])):
        Ulinks.append(Userlinks[i][j])
print(len(Ulinks), Ulinks)

3471 ['https://discuss.ardupilot.org/u/leking2T15:56:12Z', '0', 'https://discuss.ardupilot.org/u/seanhT04:42:15Z', 'https://discuss.ardupilot.org/u/fireflyer451T18:39:53Z', 'https://discuss.ardupilot.org/u/Mohd_AbuRezeqT08:32:12Z', 'https://discuss.ardupilot.org/u/StevieReyesT14:37:23Z', 'https://discuss.ardupilot.org/u/LeonardthallT07:32:34Z', 'https://discuss.ardupilot.org/u/mbolandT11:41:53Z', 'https://discuss.ardupilot.org/u/chuckntaylorT14:10:25Z', 'https://discuss.ardupilot.org/u/LeonardthallT08:02:21Z', 'https://discuss.ardupilot.org/u/LeonardthallT08:03:15Z', 'https://discuss.ardupilot.org/u/chuckntaylorT15:25:27Z', 'https://discuss.ardupilot.org/u/mbolandT19:29:34Z', 'https://discuss.ardupilot.org/u/cookmangrT17:12:56Z', 'https://discuss.ardupilot.org/u/chuckntaylorT02:16:00Z', 'https://discuss.ardupilot.org/u/cookmangrT10:40:19Z', 'https://discuss.ardupilot.org/u/chuckntaylorT16:54:31Z', 'https://discuss.ardupilot.org/u/rohanrautT13:50:17Z', 'https://discuss.ardupilot.org/u/N

In [148]:
Ulinks[0][:-10]

'https://discuss.ardupilot.org/u/leking2'

In [149]:
m_Userlinks=[]      
for Ulink in Ulinks:
    if not Ulink == '0':
        m_Userlinks.append(Ulink[:-10])
    else:
        m_Userlinks.append('0')
print(len(m_Userlinks), m_Userlinks)

3471 ['https://discuss.ardupilot.org/u/leking2', '0', 'https://discuss.ardupilot.org/u/seanh', 'https://discuss.ardupilot.org/u/fireflyer451', 'https://discuss.ardupilot.org/u/Mohd_AbuRezeq', 'https://discuss.ardupilot.org/u/StevieReyes', 'https://discuss.ardupilot.org/u/Leonardthall', 'https://discuss.ardupilot.org/u/mboland', 'https://discuss.ardupilot.org/u/chuckntaylor', 'https://discuss.ardupilot.org/u/Leonardthall', 'https://discuss.ardupilot.org/u/Leonardthall', 'https://discuss.ardupilot.org/u/chuckntaylor', 'https://discuss.ardupilot.org/u/mboland', 'https://discuss.ardupilot.org/u/cookmangr', 'https://discuss.ardupilot.org/u/chuckntaylor', 'https://discuss.ardupilot.org/u/cookmangr', 'https://discuss.ardupilot.org/u/chuckntaylor', 'https://discuss.ardupilot.org/u/rohanraut', 'https://discuss.ardupilot.org/u/Nautics', 'https://discuss.ardupilot.org/u/Neuhausmatheus', 'https://discuss.ardupilot.org/u/amilcarlucas', 'https://discuss.ardupilot.org/u/jschall', 'https://discuss.ard

In [150]:
m_Userlinks[63]

'https://discuss.ardupilot.org/u/Tim_D'

In [152]:
sf_Userlinks = set(m_Userlinks)

In [153]:
print(len(sf_Userlinks), sf_Userlinks)

755 {'https://discuss.ardupilot.org/u/renoflash', 'https://discuss.ardupilot.org/u/ottoherrero', 'https://discuss.ardupilot.org/u/wat17', 'https://discuss.ardupilot.org/u/system', 'https://discuss.ardupilot.org/u/ultrafuge', 'https://discuss.ardupilot.org/u/SkyHawkDP', 'https://discuss.ardupilot.org/u/spiekey', 'https://discuss.ardupilot.org/u/dazzab', 'https://discuss.ardupilot.org/u/Inabnit', 'https://discuss.ardupilot.org/u/Tom_Gear', 'https://discuss.ardupilot.org/u/bpereiradasilva', 'https://discuss.ardupilot.org/u/llamatrails', 'https://discuss.ardupilot.org/u/coni99', 'https://discuss.ardupilot.org/u/freezer', 'https://discuss.ardupilot.org/u/seanh', 'https://discuss.ardupilot.org/u/iseries', 'https://discuss.ardupilot.org/u/camti', 'https://discuss.ardupilot.org/u/gibbedy', 'https://discuss.ardupilot.org/u/Eosbandi', 'https://discuss.ardupilot.org/u/byronyap', 'https://discuss.ardupilot.org/u/cirilo', 'https://discuss.ardupilot.org/u/fireflyer451', 'https://discuss.ardupilot.or

In [154]:
lsf_Userlinks = list(sf_Userlinks)
print(len(lsf_Userlinks), lsf_Userlinks)

755 ['https://discuss.ardupilot.org/u/renoflash', 'https://discuss.ardupilot.org/u/ottoherrero', 'https://discuss.ardupilot.org/u/wat17', 'https://discuss.ardupilot.org/u/system', 'https://discuss.ardupilot.org/u/ultrafuge', 'https://discuss.ardupilot.org/u/SkyHawkDP', 'https://discuss.ardupilot.org/u/spiekey', 'https://discuss.ardupilot.org/u/dazzab', 'https://discuss.ardupilot.org/u/Inabnit', 'https://discuss.ardupilot.org/u/Tom_Gear', 'https://discuss.ardupilot.org/u/bpereiradasilva', 'https://discuss.ardupilot.org/u/llamatrails', 'https://discuss.ardupilot.org/u/coni99', 'https://discuss.ardupilot.org/u/freezer', 'https://discuss.ardupilot.org/u/seanh', 'https://discuss.ardupilot.org/u/iseries', 'https://discuss.ardupilot.org/u/camti', 'https://discuss.ardupilot.org/u/gibbedy', 'https://discuss.ardupilot.org/u/Eosbandi', 'https://discuss.ardupilot.org/u/byronyap', 'https://discuss.ardupilot.org/u/cirilo', 'https://discuss.ardupilot.org/u/fireflyer451', 'https://discuss.ardupilot.or

In [155]:
lsf_Userlinks[0]

'https://discuss.ardupilot.org/u/renoflash'

In [156]:
lsf_Userlinks[0].split('/')[-1]

'renoflash'

In [157]:
for i in range(len(lsf_Userlinks)):
    globals()["{}".format(lsf_Userlinks[i].split('/')[-1])] = []
    globals()["{}".format(lsf_Userlinks[i].split('/')[-1])].append('1')

In [158]:
addr = lsf_Userlinks[644]
addr

'https://discuss.ardupilot.org/u/wsalopek'

In [159]:
globals()["{}".format(lsf_Userlinks[25].split('/')[-1])]=[]
globals()["{}".format(lsf_Userlinks[25].split('/')[-1])]

[]

In [160]:
driver = webdriver.Chrome(PATH)
driver.get(addr)
profiles = driver.page_source
driver.close()
profhtml = BeautifulSoup(profiles, "html.parser")

In [161]:
profhtml

<html class="desktop-view not-mobile-device text-size-normal anon no-touch discourse-no-touch" lang="en" style="--header-offset:60.0313px;"><head>
<meta charset="utf-8"/>
<title>Profile - wsalopek - ArduPilot Discourse</title>
<meta content="ArduPilot Discussion Server" name="description"/>
<meta content="2" name="discourse_theme_id"/>
<meta content="categories" name="discourse_current_homepage"/>
<link as="style" href="/stylesheets/color_definitions_base__2_8784a4b4d4a1192b7a4f66e73449b278d8f8e2d1.css?__ws=discuss.ardupilot.org" rel="preload"/>
<link as="style" href="/stylesheets/desktop_53a570bb75afde539176eddcf4b885e7cddb0b3f.css?__ws=discuss.ardupilot.org" rel="preload"/>
<link as="style" href="/stylesheets/discourse-details_53a570bb75afde539176eddcf4b885e7cddb0b3f.css?__ws=discuss.ardupilot.org" rel="preload"/>
<link as="style" href="/stylesheets/discourse-local-dates_53a570bb75afde539176eddcf4b885e7cddb0b3f.css?__ws=discuss.ardupilot.org" rel="preload"/>
<link as="style" href="/s

In [162]:
profhtml.title.get_text()

'Profile - wsalopek - ArduPilot Discourse'

In [163]:
profhtml.select('.user-profile-hidden')

[]

In [164]:
globals()["{}".format(lsf_Userlinks[25].split('/')[-1])].append(profhtml.select('ul > li.stats-days-visited > div > span')[0].get_text().strip())
globals()["{}".format(lsf_Userlinks[25].split('/')[-1])].append(profhtml.select('ul > li.stats-time-read > div > span')[0].get_text().strip())
globals()["{}".format(lsf_Userlinks[25].split('/')[-1])].append(profhtml.select('ul > li.stats-topics-entered > div > span')[0].get_text().strip())
globals()["{}".format(lsf_Userlinks[25].split('/')[-1])].append(profhtml.select('ul > li.stats-posts-read > div > span')[0].get_text().strip())
globals()["{}".format(lsf_Userlinks[25].split('/')[-1])].append(profhtml.select('ul > li.stats-likes-given > a > div > span')[0].get_text().strip())
globals()["{}".format(lsf_Userlinks[25].split('/')[-1])].append(profhtml.select('ul > li.stats-likes-received > div > span')[0].get_text().strip())
globals()["{}".format(lsf_Userlinks[25].split('/')[-1])].append(profhtml.select('ul > li.stats-topic-count > a > div > span')[0].get_text().strip())
globals()["{}".format(lsf_Userlinks[25].split('/')[-1])].append(profhtml.select('ul > li.stats-post-count > a > div > span')[0].get_text().strip())

print(globals()["{}".format(lsf_Userlinks[25].split('/')[-1])])

IndexError: list index out of range

In [165]:
for i in range(len(lsf_Userlinks)):
    addr = lsf_Userlinks[i]
    globals()["{}".format(lsf_Userlinks[i].split('/')[-1])] = []      
    if addr == '0':
        for j in range(8):
            globals()["{}".format(lsf_Userlinks[i].split('/')[-1])].append('0')
        print(i, len(globals()["{}".format(lsf_Userlinks[i].split('/')[-1])]))
        continue
        
    time.sleep(0.1)    
    driver = webdriver.Chrome(PATH)
    driver.get(addr)
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    profiles = driver.page_source
    driver.close()
    profhtml = BeautifulSoup(profiles, "html.parser")
    if profhtml.title.get_text() == 'Page Not Found - ArduPilot Discourse' or profhtml.select('.user-profile-hidden'):
        for j in range(8):
            globals()["{}".format(lsf_Userlinks[i].split('/')[-1])].append('0')
        print(i, len(globals()["{}".format(lsf_Userlinks[i].split('/')[-1])]))
        continue
        
    time.sleep(1)
    
    globals()["{}".format(lsf_Userlinks[i].split('/')[-1])].append(profhtml.select('ul > li.stats-days-visited > div > span')[0].get_text().strip())
    globals()["{}".format(lsf_Userlinks[i].split('/')[-1])].append(profhtml.select('ul > li.stats-time-read > div > span')[0].get_text().strip())
    globals()["{}".format(lsf_Userlinks[i].split('/')[-1])].append(profhtml.select('ul > li.stats-topics-entered > div > span')[0].get_text().strip())
    globals()["{}".format(lsf_Userlinks[i].split('/')[-1])].append(profhtml.select('ul > li.stats-posts-read > div > span')[0].get_text().strip())
    globals()["{}".format(lsf_Userlinks[i].split('/')[-1])].append(profhtml.select('ul > li.stats-likes-given > a > div > span')[0].get_text().strip())
    globals()["{}".format(lsf_Userlinks[i].split('/')[-1])].append(profhtml.select('ul > li.stats-likes-received > div > span')[0].get_text().strip())
    globals()["{}".format(lsf_Userlinks[i].split('/')[-1])].append(profhtml.select('ul > li.stats-topic-count > a > div > span')[0].get_text().strip())
    globals()["{}".format(lsf_Userlinks[i].split('/')[-1])].append(profhtml.select('ul > li.stats-post-count > a > div > span')[0].get_text().strip())
    
    print(i, len(globals()["{}".format(lsf_Userlinks[i].split('/')[-1])]))

0 8
1 8
2 8
3 8
4 8
5 8
6 8
7 8
8 8
9 8
10 8
11 8
12 8
13 8
14 8
15 8
16 8
17 8
18 8
19 8
20 8
21 8
22 8
23 8
24 8
25 8
26 8
27 8
28 8
29 8
30 8
31 8
32 8
33 8
34 8
35 8
36 8
37 8
38 8
39 8
40 8
41 8
42 8
43 8
44 8
45 8
46 8
47 8
48 8
49 8
50 8
51 8
52 8
53 8
54 8
55 8
56 8
57 8
58 8
59 8
60 8
61 8
62 8
63 8
64 8
65 8
66 8
67 8
68 8
69 8
70 8
71 8
72 8
73 8
74 8
75 8
76 8
77 8
78 8
79 8
80 8
81 8
82 8
83 8
84 8
85 8
86 8
87 8
88 8
89 8
90 8
91 8
92 8
93 8
94 8
95 8
96 8
97 8
98 8
99 8
100 8
101 8
102 8
103 8
104 8
105 8
106 8
107 8
108 8
109 8
110 8
111 8
112 8
113 8
114 8
115 8
116 8
117 8
118 8
119 8
120 8
121 8
122 8
123 8
124 8
125 8
126 8
127 8
128 8
129 8
130 8
131 8
132 8
133 8
134 8
135 8
136 8
137 8
138 8
139 8
140 8
141 8
142 8
143 8
144 8
145 8
146 8
147 8
148 8
149 8
150 8
151 8
152 8
153 8
154 8
155 8
156 8
157 8
158 8
159 8
160 8
161 8
162 8
163 8
164 8
165 8
166 8
167 8
168 8
169 8
170 8
171 8
172 8
173 8
174 8
175 8
176 8
177 8
178 8
179 8
180 8
181 8
182 8
183 8
184 8


In [166]:
ID=[]     
for m_Userlink in m_Userlinks:
    ID.append(m_Userlink.split('/')[-1])
print(len(ID),ID)

3471 ['leking2', '0', 'seanh', 'fireflyer451', 'Mohd_AbuRezeq', 'StevieReyes', 'Leonardthall', 'mboland', 'chuckntaylor', 'Leonardthall', 'Leonardthall', 'chuckntaylor', 'mboland', 'cookmangr', 'chuckntaylor', 'cookmangr', 'chuckntaylor', 'rohanraut', 'Nautics', 'Neuhausmatheus', 'amilcarlucas', 'jschall', 'MM-MUC', 'Mustafa_Gokce', 'ejngnng', 'amilcarlucas', 'ejngnng', 'lploumen', 'jschall', 'lploumen', 'jschall', 'lploumen', 'jschall', 'hmsavani', 'dkemxr', 'hmsavani', 'dkemxr', 'hmsavani', 'dkemxr', 'hmsavani', 'dkemxr', 'hmsavani', 'dkemxr', 'hmsavani', 'dkemxr', 'hmsavani', 'dkemxr', 'hmsavani', 'dkemxr', 'Andre-K', 'peter750', 'Andre-K', 'jschall', 'peter750', 'Dartt-V_Systems_LLC', 'jschall', 'MrNams', 'iseries', 'jeremylindsell', 'jeremylindsell', 'jeremylindsell', 'copilot', 'iseries', 'Tim_D', 'copilot', 'mboland', 'camti', 'Karoly_Ludvigh', 'Dan_Jones', 'BigTulsa', 'mapper905', 'BigTulsa', 'mapper905', 'BigTulsa', 'mapper905', 'Babak_B.Chehreh', 'mboland', 'LuisVale', 'gnitz

In [167]:
ID[0]

'leking2'

In [168]:
leking2

['2', '20m', '9', '23', '0', '0', '1', '2']

In [169]:
globals()[ID[0]]

['2', '20m', '9', '23', '0', '0', '1', '2']

In [170]:
visit=[]
read=[]
topic=[]
posts=[]
given=[]
received=[]
topic_c=[]
post_c=[]

In [171]:
for ad in ID:
    visit.append(globals()[ad][0])
    read.append(globals()[ad][1])
    topic.append(globals()[ad][2])
    posts.append(globals()[ad][3])
    given.append(globals()[ad][4])
    received.append(globals()[ad][5])
    topic_c.append(globals()[ad][6])
    post_c.append(globals()[ad][7])

In [172]:
print(len(visit), visit)

3471 ['2', '0', '0', '131', '46', '2', '1.2k', '1.8k', '14', '1.2k', '1.2k', '14', '1.8k', '389', '14', '389', '14', '88', '51', '4', '1.5k', '45', '32', '571', '187', '1.5k', '187', '0', '45', '0', '45', '0', '45', '106', '2.2k', '106', '2.2k', '106', '2.2k', '106', '2.2k', '106', '2.2k', '106', '2.2k', '106', '2.2k', '106', '2.2k', '1.3k', '0', '1.3k', '45', '0', '0', '45', '144', '2.3k', '0', '0', '0', '216', '2.3k', '222', '216', '1.8k', '404', '111', '95', '705', '14', '705', '14', '705', '14', '41', '1.8k', '1.5k', '676', '1.8k', '1.8k', '1.5k', '676', '1.5k', '676', '676', '1.5k', '676', '94', '1.9k', '2.0k', '301', '301', '2.0k', '2.2k', '301', '1.8k', '301', '301', '2.2k', '301', '2.2k', '301', '301', '2.2k', '301', '2.2k', '301', '301', '2.2k', '301', '2.2k', '301', '301', '301', '301', '1.8k', '301', '127', '301', '45', '0', '45', '0', '0', '15', '86', '2.2k', '86', '1.2k', '391', '1.2k', '391', '391', '1.2k', '391', '989', '1.8k', '1.5k', '102', '1.5k', '144', '102', '144',

In [173]:
print(len(visit), len(read), len(topic), len(posts), len(given), len(received), len(topic), len(topic_c), len(post_c))

3471 3471 3471 3471 3471 3471 3471 3471 3471


In [174]:
mf['visit'] = visit
mf

,Post,Title,Replies,Views,Comment#,Query,Contents,Len,Days,Link,visit
1,1,About the Copter Logs -- deprecated category,1,1444,1,"Do not post here, please post in the *",You should probably change this,31,945,0,2
2,2,Attention: Please download PIXHAWK logs from t...,0,2056,0,"In order to help users better, I need readable...",,0,0,0,0
3,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,1,"Aloha, I’m a grad student at the university of...",I’m interested in this as well. I’ll be doing...,278,1,0,0
4,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,2,"Aloha, I’m a grad student at the university of...",Did you figure out how/what logs to use to get...,79,172,0,131
5,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,3,"Aloha, I’m a grad student at the university of...",I am having the same problems at the moment Ha...,183,2386,1,46
...,...,...,...,...,...,...,...,...,...,...,...
3467,888,Setting Throttle Mid,2,252,1,As mentioned in this link: * at what altitude ...,Not sure why you think altitude matters. Anyth...,105,0,0,1.3k
3468,888,Setting Throttle Mid,2,252,2,As mentioned in this link: * at what altitude ...,As they didn’t mentioned any altitude thats wh...,100,0,0,0
3469,889,"3DR Y6 crash, failsafe RTL flip, logfile analy...",1,287,1,Hoping to get some expertise on deciphering my...,I should add that the copter is physically and...,704,2,0,32
3470,890,Dataflash logging and performance,1,222,1,"Hello, I just read the documentation on datafl...","Hi bds, The easy way is simply to observe perf...",598,0,0,0


In [175]:
mf['read'] = read
mf

,Post,Title,Replies,Views,Comment#,Query,Contents,Len,Days,Link,visit,read
1,1,About the Copter Logs -- deprecated category,1,1444,1,"Do not post here, please post in the *",You should probably change this,31,945,0,2,20m
2,2,Attention: Please download PIXHAWK logs from t...,0,2056,0,"In order to help users better, I need readable...",,0,0,0,0,0
3,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,1,"Aloha, I’m a grad student at the university of...",I’m interested in this as well. I’ll be doing...,278,1,0,0,< 1m
4,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,2,"Aloha, I’m a grad student at the university of...",Did you figure out how/what logs to use to get...,79,172,0,131,10h
5,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,3,"Aloha, I’m a grad student at the university of...",I am having the same problems at the moment Ha...,183,2386,1,46,4h
...,...,...,...,...,...,...,...,...,...,...,...,...
3467,888,Setting Throttle Mid,2,252,1,As mentioned in this link: * at what altitude ...,Not sure why you think altitude matters. Anyth...,105,0,0,1.3k,4d
3468,888,Setting Throttle Mid,2,252,2,As mentioned in this link: * at what altitude ...,As they didn’t mentioned any altitude thats wh...,100,0,0,0,< 1m
3469,889,"3DR Y6 crash, failsafe RTL flip, logfile analy...",1,287,1,Hoping to get some expertise on deciphering my...,I should add that the copter is physically and...,704,2,0,32,3h
3470,890,Dataflash logging and performance,1,222,1,"Hello, I just read the documentation on datafl...","Hi bds, The easy way is simply to observe perf...",598,0,0,0,< 1m


In [176]:
mf['topic'] = topic
mf

,Post,Title,Replies,Views,Comment#,Query,Contents,Len,Days,Link,visit,read,topic
1,1,About the Copter Logs -- deprecated category,1,1444,1,"Do not post here, please post in the *",You should probably change this,31,945,0,2,20m,9
2,2,Attention: Please download PIXHAWK logs from t...,0,2056,0,"In order to help users better, I need readable...",,0,0,0,0,0,0
3,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,1,"Aloha, I’m a grad student at the university of...",I’m interested in this as well. I’ll be doing...,278,1,0,0,< 1m,0
4,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,2,"Aloha, I’m a grad student at the university of...",Did you figure out how/what logs to use to get...,79,172,0,131,10h,319
5,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,3,"Aloha, I’m a grad student at the university of...",I am having the same problems at the moment Ha...,183,2386,1,46,4h,71
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3467,888,Setting Throttle Mid,2,252,1,As mentioned in this link: * at what altitude ...,Not sure why you think altitude matters. Anyth...,105,0,0,1.3k,4d,1.4k
3468,888,Setting Throttle Mid,2,252,2,As mentioned in this link: * at what altitude ...,As they didn’t mentioned any altitude thats wh...,100,0,0,0,< 1m,0
3469,889,"3DR Y6 crash, failsafe RTL flip, logfile analy...",1,287,1,Hoping to get some expertise on deciphering my...,I should add that the copter is physically and...,704,2,0,32,3h,69
3470,890,Dataflash logging and performance,1,222,1,"Hello, I just read the documentation on datafl...","Hi bds, The easy way is simply to observe perf...",598,0,0,0,< 1m,0


In [177]:
mf['posts'] = posts
mf

,Post,Title,Replies,Views,Comment#,Query,Contents,Len,Days,Link,visit,read,topic,posts
1,1,About the Copter Logs -- deprecated category,1,1444,1,"Do not post here, please post in the *",You should probably change this,31,945,0,2,20m,9,23
2,2,Attention: Please download PIXHAWK logs from t...,0,2056,0,"In order to help users better, I need readable...",,0,0,0,0,0,0,0
3,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,1,"Aloha, I’m a grad student at the university of...",I’m interested in this as well. I’ll be doing...,278,1,0,0,< 1m,0,7
4,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,2,"Aloha, I’m a grad student at the university of...",Did you figure out how/what logs to use to get...,79,172,0,131,10h,319,2.2k
5,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,3,"Aloha, I’m a grad student at the university of...",I am having the same problems at the moment Ha...,183,2386,1,46,4h,71,553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3467,888,Setting Throttle Mid,2,252,1,As mentioned in this link: * at what altitude ...,Not sure why you think altitude matters. Anyth...,105,0,0,1.3k,4d,1.4k,8.4k
3468,888,Setting Throttle Mid,2,252,2,As mentioned in this link: * at what altitude ...,As they didn’t mentioned any altitude thats wh...,100,0,0,0,< 1m,0,16
3469,889,"3DR Y6 crash, failsafe RTL flip, logfile analy...",1,287,1,Hoping to get some expertise on deciphering my...,I should add that the copter is physically and...,704,2,0,32,3h,69,553
3470,890,Dataflash logging and performance,1,222,1,"Hello, I just read the documentation on datafl...","Hi bds, The easy way is simply to observe perf...",598,0,0,0,< 1m,0,101


In [178]:
mf['given'] = given
mf

,Post,Title,Replies,Views,Comment#,Query,Contents,Len,Days,Link,visit,read,topic,posts,given
1,1,About the Copter Logs -- deprecated category,1,1444,1,"Do not post here, please post in the *",You should probably change this,31,945,0,2,20m,9,23,0
2,2,Attention: Please download PIXHAWK logs from t...,0,2056,0,"In order to help users better, I need readable...",,0,0,0,0,0,0,0,0
3,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,1,"Aloha, I’m a grad student at the university of...",I’m interested in this as well. I’ll be doing...,278,1,0,0,< 1m,0,7,0
4,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,2,"Aloha, I’m a grad student at the university of...",Did you figure out how/what logs to use to get...,79,172,0,131,10h,319,2.2k,0
5,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,3,"Aloha, I’m a grad student at the university of...",I am having the same problems at the moment Ha...,183,2386,1,46,4h,71,553,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3467,888,Setting Throttle Mid,2,252,1,As mentioned in this link: * at what altitude ...,Not sure why you think altitude matters. Anyth...,105,0,0,1.3k,4d,1.4k,8.4k,19
3468,888,Setting Throttle Mid,2,252,2,As mentioned in this link: * at what altitude ...,As they didn’t mentioned any altitude thats wh...,100,0,0,0,< 1m,0,16,0
3469,889,"3DR Y6 crash, failsafe RTL flip, logfile analy...",1,287,1,Hoping to get some expertise on deciphering my...,I should add that the copter is physically and...,704,2,0,32,3h,69,553,0
3470,890,Dataflash logging and performance,1,222,1,"Hello, I just read the documentation on datafl...","Hi bds, The easy way is simply to observe perf...",598,0,0,0,< 1m,0,101,0


In [179]:
mf['received'] = received
mf

,Post,Title,Replies,Views,Comment#,Query,Contents,Len,Days,Link,visit,read,topic,posts,given,received
1,1,About the Copter Logs -- deprecated category,1,1444,1,"Do not post here, please post in the *",You should probably change this,31,945,0,2,20m,9,23,0,0
2,2,Attention: Please download PIXHAWK logs from t...,0,2056,0,"In order to help users better, I need readable...",,0,0,0,0,0,0,0,0,0
3,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,1,"Aloha, I’m a grad student at the university of...",I’m interested in this as well. I’ll be doing...,278,1,0,0,< 1m,0,7,0,0
4,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,2,"Aloha, I’m a grad student at the university of...",Did you figure out how/what logs to use to get...,79,172,0,131,10h,319,2.2k,0,0
5,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,3,"Aloha, I’m a grad student at the university of...",I am having the same problems at the moment Ha...,183,2386,1,46,4h,71,553,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3467,888,Setting Throttle Mid,2,252,1,As mentioned in this link: * at what altitude ...,Not sure why you think altitude matters. Anyth...,105,0,0,1.3k,4d,1.4k,8.4k,19,144
3468,888,Setting Throttle Mid,2,252,2,As mentioned in this link: * at what altitude ...,As they didn’t mentioned any altitude thats wh...,100,0,0,0,< 1m,0,16,0,0
3469,889,"3DR Y6 crash, failsafe RTL flip, logfile analy...",1,287,1,Hoping to get some expertise on deciphering my...,I should add that the copter is physically and...,704,2,0,32,3h,69,553,0,0
3470,890,Dataflash logging and performance,1,222,1,"Hello, I just read the documentation on datafl...","Hi bds, The easy way is simply to observe perf...",598,0,0,0,< 1m,0,101,0,0


In [180]:
mf['topic_c'] = topic_c
mf

,Post,Title,Replies,Views,Comment#,Query,Contents,Len,Days,Link,visit,read,topic,posts,given,received,topic_c
1,1,About the Copter Logs -- deprecated category,1,1444,1,"Do not post here, please post in the *",You should probably change this,31,945,0,2,20m,9,23,0,0,1
2,2,Attention: Please download PIXHAWK logs from t...,0,2056,0,"In order to help users better, I need readable...",,0,0,0,0,0,0,0,0,0,0
3,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,1,"Aloha, I’m a grad student at the university of...",I’m interested in this as well. I’ll be doing...,278,1,0,0,< 1m,0,7,0,0,2
4,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,2,"Aloha, I’m a grad student at the university of...",Did you figure out how/what logs to use to get...,79,172,0,131,10h,319,2.2k,0,0,6
5,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,3,"Aloha, I’m a grad student at the university of...",I am having the same problems at the moment Ha...,183,2386,1,46,4h,71,553,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3467,888,Setting Throttle Mid,2,252,1,As mentioned in this link: * at what altitude ...,Not sure why you think altitude matters. Anyth...,105,0,0,1.3k,4d,1.4k,8.4k,19,144,106
3468,888,Setting Throttle Mid,2,252,2,As mentioned in this link: * at what altitude ...,As they didn’t mentioned any altitude thats wh...,100,0,0,0,< 1m,0,16,0,0,6
3469,889,"3DR Y6 crash, failsafe RTL flip, logfile analy...",1,287,1,Hoping to get some expertise on deciphering my...,I should add that the copter is physically and...,704,2,0,32,3h,69,553,0,0,5
3470,890,Dataflash logging and performance,1,222,1,"Hello, I just read the documentation on datafl...","Hi bds, The easy way is simply to observe perf...",598,0,0,0,< 1m,0,101,0,0,4


In [181]:
mf['post_c'] = post_c
mf

,Post,Title,Replies,Views,Comment#,Query,Contents,Len,Days,Link,visit,read,topic,posts,given,received,topic_c,post_c
1,1,About the Copter Logs -- deprecated category,1,1444,1,"Do not post here, please post in the *",You should probably change this,31,945,0,2,20m,9,23,0,0,1,2
2,2,Attention: Please download PIXHAWK logs from t...,0,2056,0,"In order to help users better, I need readable...",,0,0,0,0,0,0,0,0,0,0,0
3,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,1,"Aloha, I’m a grad student at the university of...",I’m interested in this as well. I’ll be doing...,278,1,0,0,< 1m,0,7,0,0,2,5
4,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,2,"Aloha, I’m a grad student at the university of...",Did you figure out how/what logs to use to get...,79,172,0,131,10h,319,2.2k,0,0,6,11
5,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,3,"Aloha, I’m a grad student at the university of...",I am having the same problems at the moment Ha...,183,2386,1,46,4h,71,553,0,0,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3467,888,Setting Throttle Mid,2,252,1,As mentioned in this link: * at what altitude ...,Not sure why you think altitude matters. Anyth...,105,0,0,1.3k,4d,1.4k,8.4k,19,144,106,1.3k
3468,888,Setting Throttle Mid,2,252,2,As mentioned in this link: * at what altitude ...,As they didn’t mentioned any altitude thats wh...,100,0,0,0,< 1m,0,16,0,0,6,10
3469,889,"3DR Y6 crash, failsafe RTL flip, logfile analy...",1,287,1,Hoping to get some expertise on deciphering my...,I should add that the copter is physically and...,704,2,0,32,3h,69,553,0,0,5,12
3470,890,Dataflash logging and performance,1,222,1,"Hello, I just read the documentation on datafl...","Hi bds, The easy way is simply to observe perf...",598,0,0,0,< 1m,0,101,0,0,4,97


In [182]:
mf.to_csv("post_text6.csv", index=False, encoding="UTF8")

In [183]:
pd.read_csv("post_text6.csv", encoding="UTF8")

,Post,Title,Replies,Views,Comment#,Query,Contents,Len,Days,Link,visit,read,topic,posts,given,received,topic_c,post_c
0,1,About the Copter Logs -- deprecated category,1,1444,1,"Do not post here, please post in the *",You should probably change this,31,945,0,2,20m,9,23,0,0,1,2
1,2,Attention: Please download PIXHAWK logs from t...,0,2056,0,"In order to help users better, I need readable...",NaN,0,0,0,0,0,0,0,0,0,0,0
2,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,1,"Aloha, I’m a grad student at the university of...",I’m interested in this as well. I’ll be doing...,278,1,0,0,< 1m,0,7,0,0,2,5
3,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,2,"Aloha, I’m a grad student at the university of...",Did you figure out how/what logs to use to get...,79,172,0,131,10h,319,2.2k,0,0,6,11
4,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,3,"Aloha, I’m a grad student at the university of...",I am having the same problems at the moment Ha...,183,2386,1,46,4h,71,553,0,0,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3466,888,Setting Throttle Mid,2,252,1,As mentioned in this link: * at what altitude ...,Not sure why you think altitude matters. Anyth...,105,0,0,1.3k,4d,1.4k,8.4k,19,144,106,1.3k
3467,888,Setting Throttle Mid,2,252,2,As mentioned in this link: * at what altitude ...,As they didn’t mentioned any altitude thats wh...,100,0,0,0,< 1m,0,16,0,0,6,10
3468,889,"3DR Y6 crash, failsafe RTL flip, logfile analy...",1,287,1,Hoping to get some expertise on deciphering my...,I should add that the copter is physically and...,704,2,0,32,3h,69,553,0,0,5,12
3469,890,Dataflash logging and performance,1,222,1,"Hello, I just read the documentation on datafl...","Hi bds, The easy way is simply to observe perf...",598,0,0,0,< 1m,0,101,0,0,4,97


In [ ]:
# Getting user ID
U=[]
for i in range(len(Userlinks)):
    u=[]
    for j in range(len(Userlinks[i])):
        u.append(Userlinks[i][j][32:-10])
    U.append(u)
print(U)